# 1. Routes

> Takes a route text for regex matching and returns matching routes in response.

## Sample request

```bash
curl 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  -H 'lan: en' \
  --data-raw '{"routetext":"210"}'
```

## Sample response

```json
{
    "data": [
        {
            "union_rowno": 2,
            "row": 1,
            "routeno": "210-A",
            "responsecode": 200,
            "routeparentid": 7426
        },
        {
            "union_rowno": 2,
            "row": 3,
            "routeno": "210-AA",
            "responsecode": 200,
            "routeparentid": 7427
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 53,
    "responsecode": 200
}
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
import datetime
from tqdm import tqdm
import geojson

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

import requests
import pandas as pd
from fastcore.all import Path
from traffic_data_bengaluru.utils import *

In [ ]:
#| hide
#| eval: false

data_directory = Path('../data/bmtc/')
data_directory.mkdir(exist_ok=True, parents=True)

# Functions

In [ ]:
#| export
def fetch_routes(pattern: str = "", sleep_duration: float = 0.1):
    """Fetches BMTC routes matching the given pattern, or all routes if pattern is empty."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"routetext": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        routes = []
        characters = string.digits + string.ascii_lowercase
        for pattern in tqdm(characters, desc = 'Fetching routes'):
            routes += fetch_routes(pattern)
        return routes

In [ ]:
#| hide
#| eval: false

routes = fetch_routes(pattern = '210-NA')
print(json.dumps(routes, indent=4))

In [ ]:
#| export
def process_routes(routes):
    """Process and clean route data, returning a DataFrame with `route_id` and `route_number`."""
    df_routes = pd.DataFrame(routes)
    df_routes = df_routes.drop_duplicates(subset=["routeno"], keep="first")

    df_routes.rename(columns = {'routeno': 'route_number', 'routeparentid': 'route_id'}, inplace=True)
    df_routes = df_routes.sort_values(by='route_id').reset_index(drop=True)
    columns = ['route_id', 'route_number']
    return df_routes[columns]

In [ ]:
#| hide
#| eval: false
df_routes = process_routes(routes)
df_routes

In [ ]:
#| export
def get_routes(data_directory: Path):
    filepath = get_latest_directory(data_directory / "cleaned" / "routes")
    routes = pd.read_csv(filepath)
    return routes

In [ ]:
#| hide
#| eval: false
df_routes = get_routes(data_directory)
print(df_routes.shape)
df_routes.head()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()